# Fine-Tune Sentence Transformer for Medical Symptom Extraction

**Purpose:** Train a sentence transformer model to better understand how patients describe symptoms in natural language.

**Dataset:**
- 200k doctor-patient conversations (ai-medical-chatbot.csv)
- 131+ canonical symptoms (symptoms.csv)

**Output:** Fine-tuned model saved to `models/medical_symptom_matcher/`

---

## Table of Contents
1. [Setup & Installation](#setup)
2. [Load Data](#load-data)
3. [Prepare Training Data](#prepare-training)
4. [Fine-Tune Model](#fine-tune)
5. [Evaluate Performance](#evaluate)
6. [Save Model](#save)
7. [Quick Test](#test)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

---
## 1. Setup & Installation <a id='setup'></a>

In [ ]:
# Install required packages
!pip install sentence-transformers torch pandas numpy scikit-learn tqdm

In [ ]:
# Google Colab Setup
import os

# Create necessary directories
!mkdir -p data models

print("📁 Directories created!")
print("\n⚠️ IMPORTANT: Upload your datasets now!")
print("   1. Click the folder icon on the left sidebar")
print("   2. Navigate to the 'data' folder")
print("   3. Upload these files:")
print("      - ai-medical-chatbot.csv")
print("      - symptoms.csv")
print("\n💡 Or use the code below to upload via dialog:")
print("\nfrom google.colab import files")
print("uploaded = files.upload()")
print("# Then move files: !mv *.csv data/")

📁 Directories created!

⚠️ IMPORTANT: Upload your datasets now!
   1. Click the folder icon on the left sidebar
   2. Navigate to the 'data' folder
   3. Upload these files:
      - ai-medical-chatbot.csv
      - symptoms.csv

💡 Or use the code below to upload via dialog:

from google.colab import files
uploaded = files.upload()
# Then move files: !mv *.csv data/


In [ ]:
# Check GPU availability
import torch

if torch.cuda.is_available():
    print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected. Training will be MUCH slower.")
    print("   Go to Runtime > Change runtime type > Select GPU")

✅ GPU Detected: Tesla T4
   Memory: 15.83 GB


In [ ]:
import pandas as pd
import numpy as np
import torch
import re
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import json
import os
from collections import defaultdict

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ Running on CPU - training will be slower but still works!")

PyTorch version: 2.8.0+cu126
CUDA available: True
CUDA device: Tesla T4


---
## 2. Load Data <a id='load-data'></a>

In [ ]:
# Load symptoms list
print("Loading symptoms...")
symptoms_df = pd.read_csv('symptoms.csv')
SYMPTOMS = symptoms_df['symptoms'].str.lower().str.strip().tolist()
print(f"✓ Loaded {len(SYMPTOMS)} symptoms")
print(f"\nFirst 10 symptoms:")
for i, symptom in enumerate(SYMPTOMS[:10], 1):
    print(f"  {i}. {symptom}")

Loading symptoms...
✓ Loaded 380 symptoms

First 10 symptoms:
  1. anxiety and nervousness
  2. depression
  3. shortness of breath
  4. depressive or psychotic symptoms
  5. sharp chest pain
  6. dizziness
  7. insomnia
  8. abnormal involuntary movements
  9. chest tightness
  10. palpitations


In [ ]:
# Load medical conversations
print("\nLoading medical conversations...")
print("⚠️ This may take a minute for 200k rows...")

conversations_df = pd.read_csv('ai-medical-chatbot.csv')
print(f"✓ Loaded {len(conversations_df):,} conversations")
print(f"\nDataset columns: {list(conversations_df.columns)}")
print(f"\nFirst conversation example:")
print(f"Description: {conversations_df.iloc[0]['Description'][:100]}...")
print(f"Patient: {conversations_df.iloc[0]['Patient'][:150]}...")
print(f"Doctor: {conversations_df.iloc[0]['Doctor'][:150]}...")


Loading medical conversations...
⚠️ This may take a minute for 200k rows...
✓ Loaded 256,916 conversations

Dataset columns: ['Description', 'Patient', 'Doctor']

First conversation example:
Description: Q. What does abutment of the nerve root mean?...
Patient: Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for an...
Doctor: Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist...


---
## 3. Prepare Training Data <a id='prepare-training'></a>

We'll create positive training pairs where:
- **Text 1:** Patient's message
- **Text 2:** Symptom name
- **Label:** 1.0 (if symptom mentioned), 0.0 (if not mentioned)

This teaches the model to recognize when patient language matches a symptom.

In [ ]:
def clean_text(text):
    """Clean and normalize text"""
    if pd.isna(text):
        return ""
    text = str(text).lower()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

def extract_symptom_mentions(patient_text, doctor_text, symptoms_list):
    """
    Find which symptoms are mentioned in the conversation
    Returns list of mentioned symptoms
    """
    patient_text = clean_text(patient_text)
    doctor_text = clean_text(doctor_text)
    combined_text = patient_text + " " + doctor_text

    mentioned_symptoms = []

    for symptom in symptoms_list:
        symptom_clean = symptom.lower().strip()

        # Check exact match
        if symptom_clean in combined_text:
            mentioned_symptoms.append(symptom)
            continue

        # Check word-by-word match (for multi-word symptoms)
        symptom_words = symptom_clean.split()
        if len(symptom_words) > 1:
            # Check if all words appear in text (allows for different word order)
            if all(word in combined_text for word in symptom_words):
                mentioned_symptoms.append(symptom)

    return mentioned_symptoms

# Test the function
test_patient = "I have a terrible headache and feel nauseous"
test_doctor = "You may have migraine. The nausea is common with headaches."
test_mentions = extract_symptom_mentions(test_patient, test_doctor, SYMPTOMS)
print(f"Test extraction: {test_mentions}")

Test extraction: ['headache', 'nausea']


In [ ]:
def create_training_examples(conversations_df, symptoms_list, max_examples=50000, sample_negatives=True):
    """
    Create training examples from conversations

    Args:
        conversations_df: DataFrame with Patient, Doctor columns
        symptoms_list: List of canonical symptoms
        max_examples: Maximum number of positive examples to create
        sample_negatives: Whether to include negative examples (no symptom match)

    Returns:
        List of InputExample objects
    """
    training_examples = []
    positive_count = 0
    negative_count = 0

    print(f"\nCreating training examples from {len(conversations_df):,} conversations...")
    print(f"This will take 5-10 minutes...\n")

    for idx, row in tqdm(conversations_df.iterrows(), total=len(conversations_df)):
        if positive_count >= max_examples:
            break

        patient_text = clean_text(row['Patient'])
        doctor_text = clean_text(row['Doctor'])

        # Skip very short messages
        if len(patient_text.split()) < 5:
            continue

        # Find mentioned symptoms
        mentioned_symptoms = extract_symptom_mentions(patient_text, doctor_text, symptoms_list)

        if mentioned_symptoms:
            # Create positive examples
            for symptom in mentioned_symptoms:
                training_examples.append(
                    InputExample(texts=[patient_text, symptom], label=1.0)
                )
                positive_count += 1

            # Create some negative examples (symptoms NOT mentioned)
            if sample_negatives and len(mentioned_symptoms) < len(symptoms_list):
                # Sample 1-2 negative symptoms per positive
                unmentioned = [s for s in symptoms_list if s not in mentioned_symptoms]
                num_negatives = min(2, len(unmentioned))
                negative_samples = np.random.choice(unmentioned, size=num_negatives, replace=False)

                for neg_symptom in negative_samples:
                    training_examples.append(
                        InputExample(texts=[patient_text, neg_symptom], label=0.0)
                    )
                    negative_count += 1

    print(f"\n✓ Created {len(training_examples):,} training examples")
    print(f"  - Positive examples: {positive_count:,}")
    print(f"  - Negative examples: {negative_count:,}")
    print(f"  - Ratio: {positive_count/negative_count:.2f}:1 (positive:negative)")

    return training_examples

In [ ]:
# Create training examples
# Adjust max_examples based on your compute resources:
# - CPU: 10,000-20,000 examples
# - GPU: 50,000+ examples

MAX_EXAMPLES = 30000  # Adjust this based on your hardware

train_examples = create_training_examples(
    conversations_df,
    SYMPTOMS,
    max_examples=MAX_EXAMPLES,
    sample_negatives=True
)


Creating training examples from 256,916 conversations...
This will take 5-10 minutes...



  6%|▋         | 16367/256916 [00:27<06:40, 600.19it/s]


✓ Created 50,019 training examples
  - Positive examples: 30,001
  - Negative examples: 20,018
  - Ratio: 1.50:1 (positive:negative)


In [ ]:
# Split into train and validation
train_data, val_data = train_test_split(train_examples, test_size=0.1, random_state=42)

print(f"\nDataset split:")
print(f"  Training: {len(train_data):,} examples")
print(f"  Validation: {len(val_data):,} examples")

# Show some examples
print(f"\n📝 Sample training examples:")
for i in range(min(3, len(train_data))):
    example = train_data[i]
    print(f"\nExample {i+1}:")
    print(f"  Patient text: {example.texts[0][:100]}...")
    print(f"  Symptom: {example.texts[1]}")
    print(f"  Label: {'MATCH ✓' if example.label == 1.0 else 'NO MATCH ✗'}")


Dataset split:
  Training: 45,017 examples
  Validation: 5,002 examples

📝 Sample training examples:

Example 1:
  Patient text: hello doctor,my 17 years old daughter had her periods last month as normal for six days. after 10 da...
  Symptom: symptoms of bladder
  Label: MATCH ✓

Example 2:
  Patient text: my daughter age 12 yr. has gone under cxl eye surgery a month back. due to lock down its not possibl...
  Symptom: ear pain
  Label: MATCH ✓

Example 3:
  Patient text: hello doctor, i am a 28 year old male who discovered this lesion on my tongue by accident while brus...
  Symptom: back pain
  Label: MATCH ✓


---
## 4. Fine-Tune Model <a id='fine-tune'></a>

We'll fine-tune the `all-MiniLM-L6-v2` model using cosine similarity loss.

In [ ]:
# Load base model
print("Loading base model...")
base_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model = SentenceTransformer(base_model_name)
print(f"✓ Loaded {base_model_name}")
print(f"\nModel embedding dimension: {model.get_sentence_embedding_dimension()}")

Loading base model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Loaded sentence-transformers/all-MiniLM-L6-v2

Model embedding dimension: 384


In [ ]:
# Create dataloaders
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=16)
val_dataloader = DataLoader(val_data, shuffle=False, batch_size=16)

# Define loss function
train_loss = losses.CosineSimilarityLoss(model)

print(f"Training setup:")
print(f"  Batch size: 16")
print(f"  Training batches: {len(train_dataloader)}")
print(f"  Validation batches: {len(val_dataloader)}")
print(f"  Loss function: CosineSimilarityLoss")

Training setup:
  Batch size: 16
  Training batches: 2814
  Validation batches: 313
  Loss function: CosineSimilarityLoss


In [ ]:
# Training configuration
NUM_EPOCHS = 3  # Usually 2-4 epochs is enough
WARMUP_STEPS = int(len(train_dataloader) * 0.1)  # 10% of training data for warm-up

print(f"\nTraining configuration:")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Warmup steps: {WARMUP_STEPS}")
print(f"  Total training steps: {len(train_dataloader) * NUM_EPOCHS}")

# Estimate training time
if torch.cuda.is_available():
    print(f"\n⏱️ Estimated time: {NUM_EPOCHS * 15}-{NUM_EPOCHS * 30} minutes on GPU")
else:
    print(f"\n⏱️ Estimated time: {NUM_EPOCHS * 60}-{NUM_EPOCHS * 120} minutes on CPU")
    print(f"   (Consider using Google Colab with GPU if this is too slow)")


Training configuration:
  Epochs: 3
  Warmup steps: 281
  Total training steps: 8442

⏱️ Estimated time: 45-90 minutes on GPU


In [ ]:
# Create output directory
os.makedirs('models', exist_ok=True)
output_path = 'models/medical_symptom_matcher'

print(f"\n🚀 Starting training...\n")
print("="*60)

# Train the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=NUM_EPOCHS,
    warmup_steps=WARMUP_STEPS,
    output_path=output_path,
    show_progress_bar=True,
    save_best_model=True
)

print("\n" + "="*60)
print(f"✓ Training complete!")
print(f"✓ Model saved to: {output_path}")


🚀 Starting training...



Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chooyangh (chooyangh-singapore-management-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.141700
1000,0.088100
1500,0.081700
2000,0.078900
2500,0.075400
3000,0.071900
3500,0.073800
4000,0.068600
4500,0.071800
5000,0.068100



✓ Training complete!
✓ Model saved to: models/medical_symptom_matcher


---
## 5. Evaluate Performance <a id='evaluate'></a>

Let's test the fine-tuned model against the base model.

In [ ]:
# Load both models for comparison
print("Loading models for comparison...")
base_model = SentenceTransformer(base_model_name)
finetuned_model = SentenceTransformer(output_path)
print("✓ Models loaded")

# Pre-compute symptom embeddings
print("\nComputing symptom embeddings...")
base_symptom_embeddings = base_model.encode(SYMPTOMS, convert_to_tensor=True, show_progress_bar=True)
finetuned_symptom_embeddings = finetuned_model.encode(SYMPTOMS, convert_to_tensor=True, show_progress_bar=True)
print("✓ Embeddings computed")

Loading models for comparison...
✓ Models loaded

Computing symptom embeddings...


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

✓ Embeddings computed


In [ ]:
from sentence_transformers import util

def extract_symptoms_comparison(text, model, symptom_embeddings, symptoms_list, threshold=0.5, top_k=5):
    """
    Extract symptoms using a given model
    """
    text_embedding = model.encode(text, convert_to_tensor=True)
    similarities = util.cos_sim(text_embedding, symptom_embeddings)[0]

    matches = []
    for idx, score in enumerate(similarities):
        if score > threshold:
            matches.append({
                'symptom': symptoms_list[idx],
                'score': float(score)
            })

    matches = sorted(matches, key=lambda x: x['score'], reverse=True)[:top_k]
    return matches

# Test cases that previously failed
test_cases = [
    "I've been sneezing all day",
    "My head hurts really bad",
    "I am coughing a lot",
    "My stomach area has been itching like crazy",
    "I can't stop sneezing and my nose is blocked",
    "Having terrible pounding in my temples and feel nauseous",
    "My back is really painful",
    "I feel dizzy and want to throw up"
]

print("\n" + "="*80)
print("COMPARISON: BASE MODEL vs FINE-TUNED MODEL")
print("="*80)

for test_text in test_cases:
    print(f"\n📝 Input: \"{test_text}\"")
    print("-" * 80)

    # Base model results
    base_results = extract_symptoms_comparison(
        test_text, base_model, base_symptom_embeddings, SYMPTOMS, threshold=0.45
    )
    print("❌ BASE MODEL:")
    if base_results:
        for r in base_results:
            print(f"   • {r['symptom']} (confidence: {r['score']:.3f})")
    else:
        print("   • No symptoms detected")

    # Fine-tuned model results
    finetuned_results = extract_symptoms_comparison(
        test_text, finetuned_model, finetuned_symptom_embeddings, SYMPTOMS, threshold=0.45
    )
    print("\n✅ FINE-TUNED MODEL:")
    if finetuned_results:
        for r in finetuned_results:
            print(f"   • {r['symptom']} (confidence: {r['score']:.3f})")
    else:
        print("   • No symptoms detected")

    # Calculate improvement
    improvement = len(finetuned_results) - len(base_results)
    if improvement > 0:
        print(f"\n💡 Improvement: +{improvement} more symptoms detected")
    elif improvement < 0:
        print(f"\n⚠️ Note: {abs(improvement)} fewer symptoms (may be more precise)")

    print()


COMPARISON: BASE MODEL vs FINE-TUNED MODEL

📝 Input: "I've been sneezing all day"
--------------------------------------------------------------------------------
❌ BASE MODEL:
   • sneezing (confidence: 0.763)

✅ FINE-TUNED MODEL:
   • sneezing (confidence: 0.981)
   • sore in nose (confidence: 0.841)
   • nasal congestion (confidence: 0.812)
   • sore throat (confidence: 0.776)
   • cough (confidence: 0.765)

💡 Improvement: +4 more symptoms detected


📝 Input: "My head hurts really bad"
--------------------------------------------------------------------------------
❌ BASE MODEL:
   • headache (confidence: 0.569)
   • hurts to breath (confidence: 0.483)
   • ache all over (confidence: 0.472)
   • frontal headache (confidence: 0.468)
   • ear pain (confidence: 0.465)

✅ FINE-TUNED MODEL:
   • symptoms of the face (confidence: 0.890)
   • pain in eye (confidence: 0.882)
   • redness in ear (confidence: 0.863)
   • neck pain (confidence: 0.846)
   • skin pain (confidence: 0.840)


📝 In

In [ ]:
# Quantitative evaluation on validation set
print("\n" + "="*80)
print("QUANTITATIVE EVALUATION ON VALIDATION SET")
print("="*80)

def evaluate_model(model, symptom_embeddings, val_examples, threshold=0.5):
    """
    Evaluate model accuracy on validation set
    """
    correct = 0
    total = len(val_examples)

    for example in tqdm(val_examples[:1000], desc="Evaluating"):  # Sample 1000 for speed
        text = example.texts[0]
        target_symptom = example.texts[1]
        true_label = example.label

        # Get prediction
        text_embedding = model.encode(text, convert_to_tensor=True)
        symptom_idx = SYMPTOMS.index(target_symptom)
        similarity = float(util.cos_sim(text_embedding, symptom_embeddings[symptom_idx]))

        predicted_label = 1.0 if similarity > threshold else 0.0

        if predicted_label == true_label:
            correct += 1

    accuracy = correct / min(1000, total)
    return accuracy

print("\nEvaluating base model...")
base_accuracy = evaluate_model(base_model, base_symptom_embeddings, val_data)
print(f"Base model accuracy: {base_accuracy:.2%}")

print("\nEvaluating fine-tuned model...")
finetuned_accuracy = evaluate_model(finetuned_model, finetuned_symptom_embeddings, val_data)
print(f"Fine-tuned model accuracy: {finetuned_accuracy:.2%}")

improvement_pct = ((finetuned_accuracy - base_accuracy) / base_accuracy) * 100
print(f"\n🎯 Improvement: {improvement_pct:+.1f}%")


QUANTITATIVE EVALUATION ON VALIDATION SET

Evaluating base model...


Evaluating: 100%|██████████| 1000/1000 [00:07<00:00, 125.76it/s]


Base model accuracy: 46.10%

Evaluating fine-tuned model...


Evaluating: 100%|██████████| 1000/1000 [00:06<00:00, 145.74it/s]

Fine-tuned model accuracy: 91.20%

🎯 Improvement: +97.8%


---
## 6. Save Model & Metadata <a id='save'></a>

In [ ]:
# Save metadata
metadata = {
    'base_model': base_model_name,
    'training_examples': len(train_data),
    'validation_examples': len(val_data),
    'num_symptoms': len(SYMPTOMS),
    'epochs': NUM_EPOCHS,
    'base_accuracy': float(base_accuracy),
    'finetuned_accuracy': float(finetuned_accuracy),
    'improvement_pct': float(improvement_pct)
}

with open(f'{output_path}/training_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✓ Metadata saved to {output_path}/training_metadata.json")
print(f"\n📦 Model package contents:")
print(f"   {output_path}/")
for item in os.listdir(output_path):
    print(f"   ├── {item}")

✓ Metadata saved to models/medical_symptom_matcher/training_metadata.json

📦 Model package contents:
   models/medical_symptom_matcher/
   ├── 1_Pooling
   ├── training_metadata.json
   ├── special_tokens_map.json
   ├── model.safetensors
   ├── config.json
   ├── README.md
   ├── 2_Normalize
   ├── modules.json
   ├── sentence_bert_config.json
   ├── config_sentence_transformers.json
   ├── tokenizer.json
   ├── vocab.txt
   ├── tokenizer_config.json


---
## 7. Quick Test Interface <a id='test'></a>

Interactive testing of your fine-tuned model.

In [ ]:
def test_symptom_extraction(text, threshold=0.45, top_k=5):
    """
    Test the fine-tuned model on custom input
    """
    print(f"\n{'='*60}")
    print(f"Input: \"{text}\"")
    print(f"{'='*60}")

    results = extract_symptoms_comparison(
        text, finetuned_model, finetuned_symptom_embeddings,
        SYMPTOMS, threshold=threshold, top_k=top_k
    )

    if results:
        print(f"\n✅ Detected {len(results)} symptom(s):\n")
        for i, r in enumerate(results, 1):
            confidence_bar = '█' * int(r['score'] * 20)
            print(f"  {i}. {r['symptom']}")
            print(f"     Confidence: {r['score']:.3f} {confidence_bar}")
    else:
        print("\n❌ No symptoms detected above threshold")
        print(f"   Try lowering threshold (currently {threshold})")

    return results

# Test it!
test_symptom_extraction("I've been feeling dizzy and have a terrible headache")
test_symptom_extraction("My throat is sore and I can't stop coughing")
test_symptom_extraction("I have chest pain and shortness of breath")


Input: "I've been feeling dizzy and have a terrible headache"

✅ Detected 5 symptom(s):

  1. headache
     Confidence: 0.960 ███████████████████
  2. dizziness
     Confidence: 0.952 ███████████████████
  3. feeling ill
     Confidence: 0.947 ██████████████████
  4. fatigue
     Confidence: 0.933 ██████████████████
  5. ache all over
     Confidence: 0.922 ██████████████████

Input: "My throat is sore and I can't stop coughing"

✅ Detected 5 symptom(s):

  1. sore throat
     Confidence: 0.959 ███████████████████
  2. throat irritation
     Confidence: 0.915 ██████████████████
  3. cough
     Confidence: 0.874 █████████████████
  4. lump in throat
     Confidence: 0.865 █████████████████
  5. sneezing
     Confidence: 0.854 █████████████████

Input: "I have chest pain and shortness of breath"

✅ Detected 5 symptom(s):

  1. shortness of breath
     Confidence: 0.934 ██████████████████
  2. burning chest pain
     Confidence: 0.914 ██████████████████
  3. difficulty breathing
     Con

[{'symptom': 'shortness of breath', 'score': 0.9336739778518677},
 {'symptom': 'burning chest pain', 'score': 0.9136399030685425},
 {'symptom': 'difficulty breathing', 'score': 0.8817726969718933},
 {'symptom': 'chest tightness', 'score': 0.8593177795410156},
 {'symptom': 'sharp chest pain', 'score': 0.8490051031112671}]

In [ ]:
# Interactive testing (optional - uncomment to use)
# while True:
#     user_input = input("\nDescribe your symptoms (or 'quit' to exit): ")
#     if user_input.lower() in ['quit', 'exit', 'q']:
#         break
#     test_symptom_extraction(user_input)

---
## ✅ Training Complete!

### Next Steps:

1. **Your fine-tuned model is saved at:** `models/medical_symptom_matcher/`

2. **To use it in your chatbot notebook:**
```python
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('models/medical_symptom_matcher')
```

3. **Integration code is ready** - just copy the `extract_symptoms_comparison()` function to your main chatbot!

### Performance Summary:
- ✅ Base model accuracy: {base_accuracy:.1%}
- ✅ Fine-tuned accuracy: {finetuned_accuracy:.1%}
- ✅ Improvement: {improvement_pct:+.1f}%

### Tips:
- Adjust `threshold` parameter to control sensitivity (lower = more symptoms detected)
- If you need better performance, train with more examples or more epochs
- The model works best with complete sentences (not just single words)

In [ ]:
import shutil
import os

folder_to_download = 'models'
output_archive_name = folder_to_download + '.zip'

# Create the zip archive
shutil.make_archive(folder_to_download, 'zip', folder_to_download)

'/content/models.zip'